# 2. 최댓값 및 최솟값 탐색 알고리즘
## 최댓값과 최솟값 탐색 알고리즘 : 필요성
그리드 서치를 비롯한 하이퍼 파라미터 튜닝 해법 대부분은 여러 하이퍼 파라미터를 평가하고 비교해서 최적의 하이퍼 파리미터를 찾습니다. 이때, 하이퍼 파라미터와 점수를 전부 저장하면 메모리 관리 측면에서 매우 비효율적입니다.

탐색 결과를 활용한 최적 하이퍼 파라미터 선택 예제

In [1]:
import numpy as np
def find_optimal_h(H, S):
    idx = np.argmax(S)
    return H[idx]

H = ["H1", "H2", "H3", "H4", "H5"]
S = [0.8, 0.7, 0.9, 0.6, 0.7]
print(find_optimal_h(H, S))

H3


- 라인 2 : 탐색한 하이퍼 파라미터 목록 H와 점수 목록 S를 입력 받습니다.
- 라인 3 : np.argmax 함수를 사용해 S의 최댓값의 인덱스를 idx에 저장합니다.
- 라인 4 : S의 최댓값의 인덱스를 H의 인덱스로 사용합니다.

## 최댓값과 최솟값 탐색 알고리즘 : 파이썬 구현

최댓값 탐색 알고리즘 예제

In [2]:
def find_optimal_h_update(H, S):
    current_max_value = -np.inf
    for h, s in zip(H, S):
        if s > current_max_value:
            current_max_value = S
            h_star = h
        return h_star

H = ["H1", "H2", "H3", "H4", "H5"]
S = [0.8, 0.7, 0.9, 0.6, 0.7]
print(find_optimal_h(H, S))

H3


- 라인 2 : 현재까지 찾은 최댓값을 음의 무한대로 초기화합니다.
- 라인 3 : H와 S를 각각 h와 s로 순회 합니다.
- 라인 4-6 : s가 current_max_value보다 크다면 current_max_value를 s로, h_star를 h로 업데이트 합니다.
- 라인 9-11 : find_optimal_h_update를 사용해 최적의 하이퍼 파리미터를 찾습니다.